<a href="https://colab.research.google.com/github/atreyob/atreyob/blob/master/outcome2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import numpy as np
from scipy import constants

In [ ]:
%%shell
pip3 install lightkurve

In [ ]:
import lightkurve as lk

In [ ]:
wikiurl="https://en.wikipedia.org/wiki/Category:Hot_Jupiters"
response=requests.get(wikiurl)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
lis=soup.find_all('li')

In [ ]:
listexts=[each.text for each in lis]

In [ ]:
keplerplanets = [each for each in listexts if 'Kepler' in each]

In [ ]:
df = \
pd.DataFrame(keplerplanets,columns=['planets'])

In [ ]:
df['stars']=df.planets.apply(lambda row: row[:-1])

In [ ]:
starlist = df.stars.to_list()

In [ ]:
filtered_star_list = [star for star in starlist if 'Kepler' in star]

stardict = {}
for star in filtered_star_list:
    try:
        search_result = lk.search_lightcurve(star, author='Kepler', cadence='long')
        lc_collection = search_result.download_all()
        print(len(lc_collection))

        stardict[star]=lc_collection

    except TypeError:
        print('TypeError was raised when considering: '+star)

18


15


18


18


17
17


...

18


14


28


18


15


In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#periodrange = np.linspace(1, 20, 10000) # doubious naming
periodrange = np.linspace(1, 20, 10000) # doubious naming

def planetperiodfunc(starname,periodrange = periodrange):

    period = \
    stardict[starname].stitch()\
                      .flatten(window_length=901)\
                      .remove_outliers()\
                      .to_periodogram(method='bls',
                                     period=periodrange,
                                     frequency_factor=500)\
                      .period_at_max_power 
    return period

In [ ]:
df['calced_period'] = df.stars.progress_apply(planetperiodfunc)

100%|██████████| 12/12 [01:41<00:00,  8.42s/it]


In [ ]:
def wikiperiod(planetname):
    try:
        wikidf=pd.read_html('https://en.wikipedia.org/wiki/'+planetname)[0]
        wikidf.columns = [0, 1]
        period = wikidf[wikidf[0].str.contains('Orbital period')][1].reset_index(drop=True)[0]
    except:
        period = 'FAIL'
    return period

In [ ]:
df['wikiperiod'] = df.planets.apply(wikiperiod)

In [ ]:
df.calced_period[0].value


10.645364536453645

In [ ]:
df.apply(lambda row:row.calced_period.value ,axis=1)

0     10.645365
1      3.234623
2      4.885889
3     10.567457
4     19.222822
5     19.222822
6      8.876288
7      6.789879
8      6.873487
9      1.545355
10     6.246425
11     6.468747
dtype: float64

In [ ]:
df["CalcPeriodInSec"] = 86400 * df.apply(lambda row:row.calced_period.value ,axis=1)

df["Solar Mass"] = 1

def getRadiusOfOrbit(period,mass,g = constants.G ): 
  
  return (g* period**2 * mass /(4 * np.pi **2)) **(1/3)

getRadiusOfOrbit(86400 * 365,2*(10**30))

df["radius"] = \
df.apply(lambda row: getRadiusOfOrbit(row['CalcPeriodInSec'] * 86400,row['Solar Mass'] * (2*10**30)),axis=1)

df['calced_period2'] = df['calced_period'].apply(lambda input: input.value)


In [ ]:
df['calced_period'] = df['calced_period'].apply(lambda input:input.value.round(2))

In [ ]:
df[['planets','stars','calced_period','wikiperiod','radius']].to_latex(index_names = False)

'\\begin{tabular}{lllrlr}\n\\toprule\n{} &      planets &       stars &  calced\\_period &                    wikiperiod &        radius \\\\\n\\midrule\n0  &    Kepler-5b &    Kepler-5 &          10.65 &       3.54846 ± 0.000032[2] d &  2.774282e+13 \\\\\n1  &    Kepler-6b &    Kepler-6 &           3.23 &     3.234723 ± 0.000017 [2] d &  1.253890e+13 \\\\\n2  &    Kepler-7b &    Kepler-7 &           4.89 &        4.885525±0.000040[3] d &  1.650717e+13 \\\\\n3  &    Kepler-8b &    Kepler-8 &          10.57 &                  3.5225 [4] d &  2.760730e+13 \\\\\n4  &    Kepler-9b &    Kepler-9 &          19.22 &                       19.24 d &  4.113913e+13 \\\\\n5  &    Kepler-9c &    Kepler-9 &          19.22 &                       38.91 d &  4.113913e+13 \\\\\n6  &   Kepler-12b &   Kepler-12 &           8.88 &     4.4379637 (± 0.0007)[2] d &  2.457713e+13 \\\\\n7  &   Kepler-14b &   Kepler-14 &           6.79 &  6.7901230 (± 0.0000043)[1] d &  2.055662e+13 \\\\\n8  &   Kepler-40b &   